Map 1: Distribution of Onshore Wind Farms in the UK
AUTHOR: BENV0093 TBPS6
DATE: January 2026

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import warnings
import os

warnings.filterwarnings('ignore')

# Path Configuration
INPUT_DIR = 'D:/BENV0093/'
OUTPUT_DIR = 'D:/BENV0093/outputs/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("Map 1: Distribution of Onshore Wind Farms in the UK")
print("Capacity > 50 MW | Classified by Development Status")
print("="*80)
print(f"\nInput directory:  {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}\n")

# Step 1: Load UK Boundary
print("Step 1: Loading UK boundary shapefile...")

try:
    uk_boundary = gpd.read_file(os.path.join(INPUT_DIR, 'UK_ITL1_Boundaries.shp'))
    print(f"  CRS: {uk_boundary.crs}")
    print(f"  EPSG: {uk_boundary.crs.to_epsg()}")
    
    if uk_boundary.crs.to_epsg() != 27700:
        uk_boundary = uk_boundary.to_crs(epsg=27700)
        print(f"  Converted to: EPSG:27700 (British National Grid)")
    
    print(f"✓ UK boundary loaded: {len(uk_boundary)} regions")
    print(f"  Bounds: {uk_boundary.total_bounds}")
    
except Exception as e:
    print(f"✗ Error loading UK boundary: {e}")
    uk_boundary = None

# Step 2: Load and Filter Wind Farm Data
print("\nStep 2: Loading renewable energy planning database...")

df = pd.read_csv(os.path.join(INPUT_DIR, 'repd_data.csv'), encoding='latin-1', low_memory=False)
print(f"✓ Total records loaded: {len(df):,}")

onshore_wind = df[df['Technology Type'] == 'Wind Onshore'].copy()
print(f"✓ Onshore wind farms: {len(onshore_wind):,}")

onshore_wind['Installed Capacity (MWelec)'] = pd.to_numeric(
    onshore_wind['Installed Capacity (MWelec)'], 
    errors='coerce'
)

onshore_wind = onshore_wind[onshore_wind['Installed Capacity (MWelec)'] > 50].copy()
print(f"✓ Onshore wind farms > 50 MW: {len(onshore_wind):,}")

# Step 3: Classify Development Status
print("\nStep 3: Classifying development status (using 'Development Status (short)')...")

def classify_status(status):
    if pd.isna(status):
        return None
    
    status = str(status).strip()
    
    if status in ['Operational', 'Decommissioned']:
        return 'Operational'
    elif status == 'Under Construction':
        return 'Under Construction'
    elif status in ['Awaiting Construction', 'Application Submitted', 
                    'Appeal Lodged', 'No Application Required']:
        return 'Planned'
    else:
        return None

onshore_wind['Status_Category'] = onshore_wind['Development Status (short)'].apply(classify_status)
onshore_wind = onshore_wind[onshore_wind['Status_Category'].notna()].copy()
print(f"✓ Valid wind farms after classification: {len(onshore_wind):,}")

print("\n  Distribution by development status:")
status_counts = onshore_wind['Status_Category'].value_counts().sort_index()
for status, count in status_counts.items():
    capacity = onshore_wind[onshore_wind['Status_Category']==status]['Installed Capacity (MWelec)'].sum()
    print(f"    {status:20s}: {count:3d} farms ({capacity:8.1f} MW)")

# Step 4: Create GeoDataFrame
print("\nStep 4: Processing spatial coordinates...")

onshore_wind = onshore_wind[
    (onshore_wind['X-coordinate'].notna()) & 
    (onshore_wind['Y-coordinate'].notna())
].copy()

onshore_wind['X-coordinate'] = pd.to_numeric(onshore_wind['X-coordinate'], errors='coerce')
onshore_wind['Y-coordinate'] = pd.to_numeric(onshore_wind['Y-coordinate'], errors='coerce')

onshore_wind = onshore_wind[
    (onshore_wind['X-coordinate'].notna()) & 
    (onshore_wind['Y-coordinate'].notna())
].copy()

print(f"✓ Wind farms with valid coordinates: {len(onshore_wind):,}")

geometry = [
    Point(xy) for xy in zip(
        onshore_wind['X-coordinate'], 
        onshore_wind['Y-coordinate']
    )
]

wind_gdf = gpd.GeoDataFrame(
    onshore_wind, 
    geometry=geometry,
    crs='EPSG:27700'
)

print(f"  CRS: EPSG:27700 (British National Grid)")

wind_gdf['easting'] = wind_gdf.geometry.x
wind_gdf['northing'] = wind_gdf.geometry.y

wind_gdf_wgs84 = wind_gdf.to_crs(epsg=4326)
wind_gdf['longitude'] = wind_gdf_wgs84.geometry.x
wind_gdf['latitude'] = wind_gdf_wgs84.geometry.y

print(f"✓ Keeping British National Grid projection for accurate display")
print(f"  Easting range:  {wind_gdf['easting'].min():,.0f} to {wind_gdf['easting'].max():,.0f} m")
print(f"  Northing range: {wind_gdf['northing'].min():,.0f} to {wind_gdf['northing'].max():,.0f} m")

# Step 5: Create Map Visualization
print("\nStep 5: Creating map visualization...")

fig, ax = plt.subplots(1, 1, figsize=(14, 18))
ax.set_facecolor('#D6EAF8')

if uk_boundary is not None:
    uk_boundary.plot(
        ax=ax,
        color='#FFF8DC',
        edgecolor='#616161',
        linewidth=0.8,
        alpha=1.0,
        zorder=1
    )
    print("✓ UK boundary plotted")

colors = {
    'Operational': '#2E7D32',
    'Under Construction': '#FF6F00',
    'Planned': '#1565C0'
}

markers = {
    'Operational': 'o',
    'Under Construction': 's',
    'Planned': '^'
}

def calculate_marker_size(capacity, min_cap=50, max_cap=500, min_size=50, max_size=500):
    capacity = np.clip(capacity, min_cap, max_cap)
    normalized = (capacity - min_cap) / (max_cap - min_cap)
    size = min_size + normalized * (max_size - min_size)
    return size

for status in ['Operational', 'Under Construction', 'Planned']:
    subset = wind_gdf[wind_gdf['Status_Category'] == status]
    
    if len(subset) > 0:
        sizes = subset['Installed Capacity (MWelec)'].apply(calculate_marker_size)
        
        subset.plot(
            ax=ax,
            markersize=sizes,
            color=colors[status],
            marker=markers[status],
            alpha=0.75,
            edgecolor='black',
            linewidth=0.6,
            label=f'{status} (n={len(subset)})',
            zorder=5,
            aspect='equal'
        )

ax.set_xlabel('Easting (km)', fontsize=13, fontweight='bold')
ax.set_ylabel('Northing (km)', fontsize=13, fontweight='bold')
ax.set_title(
    'Distribution of Onshore Wind Farms in the United Kingdom\n' +
    'Classified by Development Status (Installed Capacity > 50 MW)\n' +
    'Projection: British National Grid (EPSG:27700)',
    fontsize=16,
    fontweight='bold',
    pad=25
)

ax.set_xlim(0, 700000)
ax.set_ylim(0, 1300000)
ax.grid(True, alpha=0.25, linestyle='-', linewidth=0.5, color='gray', zorder=0)
ax.set_aspect('equal', adjustable='box')
ax.tick_params(axis='both', which='major', labelsize=10)

from matplotlib.ticker import FuncFormatter

def format_meters_to_km(value, tick_number):
    return f'{int(value/1000)}'

ax.xaxis.set_major_formatter(FuncFormatter(format_meters_to_km))
ax.yaxis.set_major_formatter(FuncFormatter(format_meters_to_km))

# Legends
status_legend = ax.legend(
    loc='lower left',
    bbox_to_anchor=(0.02, 0.15),
    title='Development Status',
    title_fontsize=12,
    fontsize=11,
    frameon=True,
    fancybox=True,
    shadow=True,
    framealpha=0.95,
    markerscale=1.8,
    edgecolor='black'
)
ax.add_artist(status_legend)

size_handles = []
size_labels = []
for cap in [50, 150, 300, 500]:
    handle = plt.scatter(
        [], [], 
        s=calculate_marker_size(cap), 
        c='gray',
        marker='o',
        edgecolors='black', 
        linewidth=0.6, 
        alpha=0.7
    )
    size_handles.append(handle)
    size_labels.append(f'{cap} MW')

size_legend = ax.legend(
    handles=size_handles,
    labels=size_labels,
    loc='lower left',
    bbox_to_anchor=(0.02, 0.02),
    title='Installed Capacity',
    title_fontsize=12,
    fontsize=11,
    frameon=True,
    fancybox=True,
    shadow=True,
    framealpha=0.95,
    edgecolor='black'
)

# North Arrow and Scale Bar
north_arrow_x = 0.95
north_arrow_y = 0.95

arrow_props = dict(
    arrowstyle='->,head_width=0.6,head_length=0.8',
    lw=2.5,
    color='black'
)

ax.annotate(
    '',
    xy=(north_arrow_x, north_arrow_y),
    xytext=(north_arrow_x, north_arrow_y - 0.05),
    xycoords='axes fraction',
    arrowprops=arrow_props,
    zorder=15
)

ax.text(
    north_arrow_x, north_arrow_y + 0.015,
    'N',
    transform=ax.transAxes,
    fontsize=16,
    fontweight='bold',
    horizontalalignment='center',
    verticalalignment='bottom',
    zorder=15
)

scale_km = 100
scale_x_start = 0.70
scale_x_end = scale_x_start + 0.18
scale_y = 0.08

ax.plot(
    [scale_x_start, scale_x_end],
    [scale_y, scale_y],
    transform=ax.transAxes,
    color='black',
    linewidth=3,
    zorder=15
)

for x in [scale_x_start, scale_x_end]:
    ax.plot(
        [x, x],
        [scale_y - 0.005, scale_y + 0.005],
        transform=ax.transAxes,
        color='black',
        linewidth=3,
        zorder=15
    )

ax.text(
    (scale_x_start + scale_x_end) / 2,
    scale_y - 0.015,
    f'{scale_km} km',
    transform=ax.transAxes,
    fontsize=11,
    fontweight='bold',
    horizontalalignment='center',
    verticalalignment='top',
    zorder=15
)

print("✓ North arrow and scale bar added")

plt.tight_layout()

# Step 6: Save Outputs
print("\nStep 6: Saving outputs...")

total_farms = len(wind_gdf)
total_capacity = wind_gdf['Installed Capacity (MWelec)'].sum()
avg_capacity = wind_gdf['Installed Capacity (MWelec)'].mean()
n_operational = len(wind_gdf[wind_gdf['Status_Category'] == 'Operational'])
n_construction = len(wind_gdf[wind_gdf['Status_Category'] == 'Under Construction'])
n_planned = len(wind_gdf[wind_gdf['Status_Category'] == 'Planned'])

print("\n" + "-"*80)
print("MAP STATISTICS:")
print("-"*80)
print(f"  Total Wind Farms:        {total_farms:3d}")
print(f"    • Operational:         {n_operational:3d}")
print(f"    • Under Construction:  {n_construction:3d}")
print(f"    • Planned:             {n_planned:3d}")
print(f"  Total Capacity:      {total_capacity:8,.0f} MW")
print(f"  Average Capacity:    {avg_capacity:8,.1f} MW")
print("-"*80)

output_png = os.path.join(OUTPUT_DIR, 'map1_onshore_wind_distribution.png')
plt.savefig(output_png, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
print(f"\n✓ PNG map saved: {output_png}")

output_pdf = os.path.join(OUTPUT_DIR, 'map1_onshore_wind_distribution.pdf')
plt.savefig(output_pdf, dpi=300, bbox_inches='tight', facecolor='white', edgecolor='none')
print(f"✓ PDF map saved: {output_pdf}")

plt.close()

# Step 7: Generate Summary Statistics
print("\nStep 7: Generating summary statistics...")

summary = wind_gdf.groupby('Status_Category').agg({
    'Installed Capacity (MWelec)': ['count', 'sum', 'mean', 'min', 'max', 'median']
}).round(2)

summary.columns = ['Count', 'Total Capacity (MW)', 'Mean Capacity (MW)', 
                   'Min Capacity (MW)', 'Max Capacity (MW)', 'Median Capacity (MW)']
summary = summary.reset_index()
summary = summary.sort_values('Total Capacity (MW)', ascending=False)

print("\n" + "="*80)
print("SUMMARY STATISTICS: Onshore Wind Farms > 50 MW")
print("="*80)
print(summary.to_string(index=False))
print("="*80)

summary_csv = os.path.join(OUTPUT_DIR, 'map1_onshore_wind_summary.csv')
summary.to_csv(summary_csv, index=False)
print(f"\n✓ Summary CSV saved: {summary_csv}")

output_columns = [
    'Site Name', 
    'Operator (or Applicant)', 
    'Status_Category',
    'Development Status (short)',
    'Installed Capacity (MWelec)', 
    'County', 
    'Region',
    'X-coordinate',
    'Y-coordinate',
    'longitude', 
    'latitude'
]

wind_data_csv = os.path.join(OUTPUT_DIR, 'onshore_wind_filtered_50MW.csv')
wind_gdf[output_columns].to_csv(wind_data_csv, index=False)
print(f"✓ Detailed data CSV saved: {wind_data_csv}")

wind_geojson = os.path.join(OUTPUT_DIR, 'onshore_wind_filtered_50MW.geojson')
wind_gdf[['Site Name', 'Status_Category', 'Installed Capacity (MWelec)', 'geometry']].to_file(
    wind_geojson, 
    driver='GeoJSON'
)
print(f"✓ GeoJSON saved: {wind_geojson}")

print("\n" + "="*80)
print("✓ Map 1 generation completed successfully!")
print("="*80)
print(f"\nOutputs saved to: {OUTPUT_DIR}")
print("  • map1_onshore_wind_distribution.png")
print("  • map1_onshore_wind_distribution.pdf")
print("  • map1_onshore_wind_summary.csv")
print("  • onshore_wind_filtered_50MW.csv")
print("  • onshore_wind_filtered_50MW.geojson")
print("\n" + "="*80)

Map 1: Distribution of Onshore Wind Farms in the UK
Capacity > 50 MW | Classified by Development Status

Input directory:  D:/BENV0093/
Output directory: D:/BENV0093/outputs/

Step 1: Loading UK boundary shapefile...
  CRS: EPSG:27700
  EPSG: 27700
✓ UK boundary loaded: 12 regions
  Bounds: [-1.16192800e+02  7.05409980e+03  6.55644798e+05  1.21861644e+06]

Step 2: Loading renewable energy planning database...
✓ Total records loaded: 13,524
✓ Onshore wind farms: 2,657
✓ Onshore wind farms > 50 MW: 315

Step 3: Classifying development status (using 'Development Status (short)')...
✓ Valid wind farms after classification: 205

  Distribution by development status:
    Operational         :  59 farms (  6172.9 MW)
    Planned             : 133 farms ( 13924.1 MW)
    Under Construction  :  13 farms (  1395.8 MW)

Step 4: Processing spatial coordinates...
✓ Wind farms with valid coordinates: 202
  CRS: EPSG:27700 (British National Grid)
✓ Keeping British National Grid projection for accurat

Map 2: Regional Onshore Wind Capacity Distribution (Operational Only)

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from shapely.geometry import Point
import warnings
import os

warnings.filterwarnings('ignore')

# Path Configuration
INPUT_DIR = 'D:/BENV0093/'
OUTPUT_DIR = 'D:/BENV0093/outputs/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("Map 2: Regional Onshore Wind Capacity Distribution")
print("Showing Total and Operational Capacity by UK Region")
print("="*80)
print(f"\nInput directory:  {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}\n")

# Step 1: Load UK Regional Boundaries
print("Step 1: Loading UK ITL1 regional boundaries...")

try:
    uk_regions = gpd.read_file(os.path.join(INPUT_DIR, 'UK_ITL1_Boundaries.shp'))
    
    if uk_regions.crs.to_epsg() != 27700:
        uk_regions = uk_regions.to_crs(epsg=27700)
    
    print(f"✓ UK regions loaded: {len(uk_regions)} regions")
    print(f"  CRS: EPSG:27700 (British National Grid)")
    
    if 'ITL121NM' in uk_regions.columns:
        region_col = 'ITL121NM'
    elif 'ITL121CD' in uk_regions.columns:
        region_col = 'ITL121CD'
    else:
        name_cols = [col for col in uk_regions.columns if 'NM' in col.upper() or 'NAME' in col.upper()]
        region_col = name_cols[0] if name_cols else uk_regions.columns[0]
    
    print(f"  Region identifier column: {region_col}")
    print(f"  Regions: {', '.join(uk_regions[region_col].tolist()[:5])}...")
    
except Exception as e:
    print(f"✗ Error loading regional boundaries: {e}")
    exit(1)

# Step 2: Load Wind Farm Data
print("\nStep 2: Loading renewable energy planning database...")

df = pd.read_csv(os.path.join(INPUT_DIR, 'repd_data.csv'), encoding='latin-1', low_memory=False)
print(f"✓ Total records loaded: {len(df):,}")

onshore_wind = df[df['Technology Type'] == 'Wind Onshore'].copy()
print(f"✓ Onshore wind farms: {len(onshore_wind):,}")

onshore_wind['Installed Capacity (MWelec)'] = pd.to_numeric(
    onshore_wind['Installed Capacity (MWelec)'], 
    errors='coerce'
)

onshore_wind = onshore_wind[onshore_wind['Installed Capacity (MWelec)'].notna()].copy()
print(f"✓ Wind farms with valid capacity: {len(onshore_wind):,}")

# Step 3: Classify Operational Status
print("\nStep 3: Classifying operational status...")

def is_operational(status):
    if pd.isna(status):
        return False
    status = str(status).strip()
    return status in ['Operational', 'Decommissioned']

onshore_wind['Is_Operational'] = onshore_wind['Development Status (short)'].apply(is_operational)

n_operational = onshore_wind['Is_Operational'].sum()
print(f"✓ Operational wind farms: {n_operational:,}")
print(f"  Other statuses: {len(onshore_wind) - n_operational:,}")

# Step 4: Spatial Join
print("\nStep 4: Performing spatial join to assign wind farms to regions...")

onshore_wind = onshore_wind[
    (onshore_wind['X-coordinate'].notna()) & 
    (onshore_wind['Y-coordinate'].notna())
].copy()

onshore_wind['X-coordinate'] = pd.to_numeric(onshore_wind['X-coordinate'], errors='coerce')
onshore_wind['Y-coordinate'] = pd.to_numeric(onshore_wind['Y-coordinate'], errors='coerce')

onshore_wind = onshore_wind[
    (onshore_wind['X-coordinate'].notna()) & 
    (onshore_wind['Y-coordinate'].notna())
].copy()

print(f"✓ Wind farms with valid coordinates: {len(onshore_wind):,}")

geometry = [Point(xy) for xy in zip(onshore_wind['X-coordinate'], onshore_wind['Y-coordinate'])]
wind_gdf = gpd.GeoDataFrame(onshore_wind, geometry=geometry, crs='EPSG:27700')

print(f"✓ GeoDataFrame created in EPSG:27700")

wind_with_regions = gpd.sjoin(wind_gdf, uk_regions, how='left', predicate='within')

print(f"✓ Spatial join completed")
print(f"  Wind farms matched to regions: {wind_with_regions[region_col].notna().sum():,}")
print(f"  Wind farms without region match: {wind_with_regions[region_col].isna().sum():,}")

# Step 5: Aggregate Operational Wind Farms by Region
print("\nStep 5: Aggregating operational wind farms by region...")

operational_wind = wind_with_regions[wind_with_regions['Is_Operational'] == True].copy()
print(f"✓ Filtering operational wind farms: {len(operational_wind):,}")

regional_operational = operational_wind.groupby(region_col).agg({
    'Installed Capacity (MWelec)': 'sum',
    'Site Name': 'count'
}).reset_index()
regional_operational.columns = [region_col, 'Operational_Capacity_MW', 'Operational_Count']

regional_operational['Operational_Capacity_GW'] = regional_operational['Operational_Capacity_MW'] / 1000

uk_regions_with_data = uk_regions.merge(regional_operational, on=region_col, how='left')
uk_regions_with_data['Operational_Capacity_GW'].fillna(0, inplace=True)
uk_regions_with_data['Operational_Count'].fillna(0, inplace=True)
uk_regions_with_data['Operational_Count'] = uk_regions_with_data['Operational_Count'].astype(int)

print(f"✓ Regional aggregation completed (operational only)")
print(f"\nTop 5 regions by operational capacity:")
top5 = regional_operational.nlargest(5, 'Operational_Capacity_GW')
for _, row in top5.iterrows():
    print(f"  {row[region_col]}: {row['Operational_Capacity_GW']:.2f} GW ({int(row['Operational_Count'])} farms)")

# Step 6: Create Choropleth Map
print("\nStep 6: Creating choropleth map...")

fig, ax = plt.subplots(1, 1, figsize=(12, 16))
ax.set_facecolor('#D6EAF8')

uk_regions_with_data.plot(
    ax=ax,
    column='Operational_Capacity_GW',
    cmap='YlGn',
    edgecolor='#424242',
    linewidth=0.8,
    legend=True,
    legend_kwds={
        'label': 'Operational Capacity (GW)',
        'orientation': 'vertical',
        'shrink': 0.6,
        'aspect': 20,
        'pad': 0.05
    },
    missing_kwds={'color': '#F5F5DC', 'label': 'No Data'},
    zorder=1
)

print("✓ Choropleth map plotted")

# Step 7: Add Region Labels
print("Step 7: Adding region labels...")

for idx, row in uk_regions_with_data.iterrows():
    centroid = row.geometry.centroid
    
    region_name = row[region_col]
    operational_cap = row['Operational_Capacity_GW']
    operational_count = int(row['Operational_Count'])
    
    if operational_cap > 0:
        label_text = f"{operational_cap:.2f} GW\n({operational_count})"
        text_color = 'black' if operational_cap < 3 else 'white'
        
        ax.text(
            centroid.x, centroid.y,
            label_text,
            fontsize=9,
            fontweight='bold',
            ha='center',
            va='center',
            color=text_color,
            bbox=dict(
                boxstyle='round,pad=0.3',
                facecolor='white' if operational_cap < 3 else 'black',
                edgecolor='none',
                alpha=0.7
            ),
            zorder=10
        )

print("✓ Region labels added")

# Customize Plot
ax.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax.set_title(
    'Regional Installed Capacity — Onshore Wind (Operational)\n' +
    'Operational Capacity in GW and (Number of Operational Farms)\n' +
    'Projection: British National Grid (EPSG:27700)',
    fontsize=14,
    fontweight='bold',
    pad=20
)

ax.set_xlim(0, 700000)
ax.set_ylim(0, 1300000)
ax.grid(True, alpha=0.2, linestyle='-', linewidth=0.5, color='gray', zorder=0)
ax.set_aspect('equal', adjustable='box')

from matplotlib.ticker import FuncFormatter

def format_meters_to_km(value, tick_number):
    return f'{int(value/1000)}'

ax.xaxis.set_major_formatter(FuncFormatter(format_meters_to_km))
ax.yaxis.set_major_formatter(FuncFormatter(format_meters_to_km))

# North Arrow
north_arrow_x = 0.95
north_arrow_y = 0.95

arrow_props = dict(
    arrowstyle='->,head_width=0.6,head_length=0.8',
    lw=2.5,
    color='black'
)

ax.annotate(
    '',
    xy=(north_arrow_x, north_arrow_y),
    xytext=(north_arrow_x, north_arrow_y - 0.05),
    xycoords='axes fraction',
    arrowprops=arrow_props,
    zorder=15
)

ax.text(
    north_arrow_x, north_arrow_y + 0.015,
    'N',
    transform=ax.transAxes,
    fontsize=16,
    fontweight='bold',
    ha='center',
    va='bottom',
    zorder=15
)

# Scale Bar
scale_x_start = 0.05
scale_x_end = scale_x_start + 0.15
scale_y = 0.05

ax.plot(
    [scale_x_start, scale_x_end],
    [scale_y, scale_y],
    transform=ax.transAxes,
    color='black',
    linewidth=3,
    zorder=15
)

for x in [scale_x_start, scale_x_end]:
    ax.plot(
        [x, x],
        [scale_y - 0.005, scale_y + 0.005],
        transform=ax.transAxes,
        color='black',
        linewidth=3,
        zorder=15
    )

ax.text(
    (scale_x_start + scale_x_end) / 2,
    scale_y - 0.015,
    '100 km',
    transform=ax.transAxes,
    fontsize=10,
    fontweight='bold',
    ha='center',
    va='top',
    zorder=15
)

print("✓ North arrow and scale bar added")

plt.tight_layout()

# Step 8: Save Outputs
print("\nStep 8: Saving outputs...")

output_png = os.path.join(OUTPUT_DIR, 'map2_regional_onshore_capacity.png')
plt.savefig(output_png, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ PNG map saved: {output_png}")

output_pdf = os.path.join(OUTPUT_DIR, 'map2_regional_onshore_capacity.pdf')
plt.savefig(output_pdf, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ PDF map saved: {output_pdf}")

plt.close()

# Step 9: Export Summary Table
print("\nStep 9: Generating summary table...")

regional_summary = regional_operational.sort_values('Operational_Capacity_GW', ascending=False)

total_uk_operational = regional_summary['Operational_Capacity_GW'].sum()
regional_summary['Percentage_of_UK_Operational'] = (
    regional_summary['Operational_Capacity_GW'] / total_uk_operational * 100
).round(2)

output_summary = regional_summary[[
    region_col,
    'Operational_Capacity_GW',
    'Operational_Count',
    'Percentage_of_UK_Operational'
]].copy()

output_summary.columns = [
    'Region',
    'Operational Capacity (GW)',
    'Number of Operational Farms',
    '% of UK Operational Capacity'
]

print("\n" + "="*80)
print("REGIONAL SUMMARY - Onshore Wind (Operational Only)")
print("="*80)
print(output_summary.to_string(index=False))
print("="*80)
print(f"\nUK Total Operational Capacity: {total_uk_operational:.2f} GW")
print(f"UK Total Operational Wind Farms: {regional_summary['Operational_Count'].sum():.0f}")
print("="*80)

summary_csv = os.path.join(OUTPUT_DIR, 'map2_regional_summary.csv')
output_summary.to_csv(summary_csv, index=False)
print(f"\n✓ Summary table saved: {summary_csv}")

print("\n" + "="*80)
print("✓ Map 2 generation completed successfully!")
print("="*80)
print(f"\nOutputs saved to: {OUTPUT_DIR}")
print("  • map2_regional_onshore_capacity.png")
print("  • map2_regional_onshore_capacity.pdf")
print("  • map2_regional_summary.csv")
print("\n" + "="*80)

Map 2: Regional Onshore Wind Capacity Distribution
Showing Total and Operational Capacity by UK Region

Input directory:  D:/BENV0093/
Output directory: D:/BENV0093/outputs/

Step 1: Loading UK ITL1 regional boundaries...
✓ UK regions loaded: 12 regions
  CRS: EPSG:27700 (British National Grid)
  Region identifier column: ITL121NM
  Regions: North East (England), North West (England), Yorkshire and The Humber, East Midlands (England), West Midlands (England)...

Step 2: Loading renewable energy planning database...
✓ Total records loaded: 13,524
✓ Onshore wind farms: 2,657
✓ Wind farms with valid capacity: 2,565

Step 3: Classifying operational status...
✓ Operational wind farms: 779
  Other statuses: 1,786

Step 4: Performing spatial join to assign wind farms to regions...
✓ Wind farms with valid coordinates: 2,561
✓ GeoDataFrame created in EPSG:27700
✓ Spatial join completed
  Wind farms matched to regions: 2,546
  Wind farms without region match: 15

Step 5: Aggregating operational 

Map 3: Spatial Density Comparison - Onshore vs Offshore Wind (Operational)

In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
import warnings
import os

warnings.filterwarnings('ignore')

# Path Configuration
INPUT_DIR = 'D:/BENV0093/'
OUTPUT_DIR = 'D:/BENV0093/outputs/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*80)
print("Map 3: Spatial Density Comparison - Onshore vs Offshore Wind")
print("Hexbin Density Analysis for Operational Wind Farms")
print("="*80)
print(f"\nInput directory:  {INPUT_DIR}")
print(f"Output directory: {OUTPUT_DIR}\n")

# Step 1: Load UK Boundary
print("Step 1: Loading UK boundary shapefile...")

try:
    uk_boundary = gpd.read_file(os.path.join(INPUT_DIR, 'UK_ITL1_Boundaries.shp'))
    
    if uk_boundary.crs.to_epsg() != 27700:
        uk_boundary = uk_boundary.to_crs(epsg=27700)
    
    uk_outline = uk_boundary.dissolve()
    
    print(f"✓ UK boundary loaded and dissolved")
    print(f"  CRS: EPSG:27700 (British National Grid)")
    
except Exception as e:
    print(f"✗ Error loading UK boundary: {e}")
    uk_outline = None

# Step 2: Load Wind Farm Data
print("\nStep 2: Loading renewable energy planning database...")

df = pd.read_csv(os.path.join(INPUT_DIR, 'repd_data.csv'), encoding='latin-1', low_memory=False)
print(f"✓ Total records loaded: {len(df):,}")

wind_farms = df[df['Technology Type'].isin(['Wind Onshore', 'Wind Offshore'])].copy()
print(f"✓ Wind farms (onshore + offshore): {len(wind_farms):,}")

wind_farms['Installed Capacity (MWelec)'] = pd.to_numeric(
    wind_farms['Installed Capacity (MWelec)'], 
    errors='coerce'
)

# Step 3: Filter Operational Wind Farms
print("\nStep 3: Filtering operational wind farms...")

def is_operational(status):
    if pd.isna(status):
        return False
    status = str(status).strip()
    return status in ['Operational', 'Decommissioned']

wind_farms['Is_Operational'] = wind_farms['Development Status (short)'].apply(is_operational)

operational_wind = wind_farms[wind_farms['Is_Operational'] == True].copy()
print(f"✓ Operational wind farms: {len(operational_wind):,}")

onshore = operational_wind[operational_wind['Technology Type'] == 'Wind Onshore'].copy()
offshore = operational_wind[operational_wind['Technology Type'] == 'Wind Offshore'].copy()

print(f"  Onshore operational: {len(onshore):,}")
print(f"  Offshore operational: {len(offshore):,}")

# Step 4: Process Coordinates
print("\nStep 4: Processing spatial coordinates...")

def process_coordinates(df, tech_type):
    df_clean = df[
        (df['X-coordinate'].notna()) & 
        (df['Y-coordinate'].notna())
    ].copy()
    
    df_clean['X-coordinate'] = pd.to_numeric(df_clean['X-coordinate'], errors='coerce')
    df_clean['Y-coordinate'] = pd.to_numeric(df_clean['Y-coordinate'], errors='coerce')
    
    df_clean = df_clean[
        (df_clean['X-coordinate'].notna()) & 
        (df_clean['Y-coordinate'].notna())
    ].copy()
    
    geometry = [Point(xy) for xy in zip(df_clean['X-coordinate'], df_clean['Y-coordinate'])]
    gdf = gpd.GeoDataFrame(df_clean, geometry=geometry, crs='EPSG:27700')
    
    print(f"  {tech_type}: {len(gdf):,} with valid coordinates")
    return gdf

onshore_gdf = process_coordinates(onshore, 'Onshore')
offshore_gdf = process_coordinates(offshore, 'Offshore')

# Step 5: Create Hexbin Density Maps
print("\nStep 5: Creating hexbin density comparison maps...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 12))

gridsize = 40
cmap = 'plasma'
vmin = 1

if len(onshore_gdf) > 0:
    onshore_x = onshore_gdf.geometry.x.values
    onshore_y = onshore_gdf.geometry.y.values

if len(offshore_gdf) > 0:
    offshore_x = offshore_gdf.geometry.x.values
    offshore_y = offshore_gdf.geometry.y.values

# Left Panel: Onshore
if uk_outline is not None:
    uk_outline.plot(ax=ax1, color='white', edgecolor='#424242', linewidth=1, alpha=0.8, zorder=1)

if len(onshore_gdf) > 0:
    from matplotlib.colors import LogNorm
    
    hexbin1 = ax1.hexbin(
        onshore_gdf.geometry.x,
        onshore_gdf.geometry.y,
        gridsize=gridsize,
        cmap=cmap,
        mincnt=1,
        edgecolors='none',
        alpha=0.85,
        norm=LogNorm(vmin=1, vmax=50),
        linewidths=0.2,
        zorder=2
    )
    
    cbar1 = plt.colorbar(hexbin1, ax=ax1, fraction=0.046, pad=0.04)
    cbar1.set_label('Number of wind farms per hexagon', fontsize=11, fontweight='bold')
    cbar1.ax.tick_params(labelsize=9)

ax1.set_facecolor('#E8F4F8')
ax1.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax1.set_title(
    'Onshore (Operational) — Hexbin density',
    fontsize=14,
    fontweight='bold',
    pad=15
)

ax1.set_xlim(0, 700000)
ax1.set_ylim(0, 1300000)
ax1.set_aspect('equal', adjustable='box')

from matplotlib.ticker import FuncFormatter

def format_km(value, tick_number):
    return f'{int(value/1000)}'

ax1.xaxis.set_major_formatter(FuncFormatter(format_km))
ax1.yaxis.set_major_formatter(FuncFormatter(format_km))

ax1.grid(True, alpha=0.2, linestyle='-', linewidth=0.5, color='gray', zorder=0)

arrow_props = dict(arrowstyle='->,head_width=0.5,head_length=0.7', lw=2, color='black')
ax1.annotate('', xy=(0.95, 0.95), xytext=(0.95, 0.90), 
             xycoords='axes fraction', arrowprops=arrow_props, zorder=15)
ax1.text(0.95, 0.96, 'N', transform=ax1.transAxes, fontsize=14, 
         fontweight='bold', ha='center', va='bottom', zorder=15)

print(f"✓ Onshore density map created (logarithmic scale)")

# Right Panel: Offshore
if uk_outline is not None:
    uk_outline.plot(ax=ax2, color='white', edgecolor='#424242', linewidth=1, alpha=0.8, zorder=1)

if len(offshore_gdf) > 0:
    hexbin2 = ax2.hexbin(
        offshore_gdf.geometry.x,
        offshore_gdf.geometry.y,
        gridsize=gridsize,
        cmap=cmap,
        mincnt=1,
        edgecolors='none',
        alpha=0.85,
        vmin=1,
        vmax=12,
        linewidths=0.2,
        zorder=2
    )
    
    cbar2 = plt.colorbar(hexbin2, ax=ax2, fraction=0.046, pad=0.04)
    cbar2.set_label('Number of wind farms per hexagon', fontsize=11, fontweight='bold')
    cbar2.ax.tick_params(labelsize=9)

ax2.set_facecolor('#E8F4F8')
ax2.set_xlabel('Easting (km)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Northing (km)', fontsize=12, fontweight='bold')
ax2.set_title(
    'Offshore (Operational) — Hexbin density',
    fontsize=14,
    fontweight='bold',
    pad=15
)

ax2.set_xlim(0, 700000)
ax2.set_ylim(0, 1300000)
ax2.set_aspect('equal', adjustable='box')

ax2.xaxis.set_major_formatter(FuncFormatter(format_km))
ax2.yaxis.set_major_formatter(FuncFormatter(format_km))

ax2.grid(True, alpha=0.2, linestyle='-', linewidth=0.5, color='gray', zorder=0)

ax2.annotate('', xy=(0.95, 0.95), xytext=(0.95, 0.90), 
             xycoords='axes fraction', arrowprops=arrow_props, zorder=15)
ax2.text(0.95, 0.96, 'N', transform=ax2.transAxes, fontsize=14, 
         fontweight='bold', ha='center', va='bottom', zorder=15)

print(f"✓ Offshore density map created (linear scale)")

# Add Scale Bars
def add_scale_bar(ax, position='lower left'):
    if position == 'lower left':
        x_start, y_pos = 0.05, 0.05
    else:
        x_start, y_pos = 0.05, 0.05
    
    x_end = x_start + 0.12
    
    ax.plot([x_start, x_end], [y_pos, y_pos], transform=ax.transAxes,
            color='black', linewidth=2.5, zorder=15)
    
    for x in [x_start, x_end]:
        ax.plot([x, x], [y_pos - 0.005, y_pos + 0.005], transform=ax.transAxes,
                color='black', linewidth=2.5, zorder=15)
    
    ax.text((x_start + x_end) / 2, y_pos - 0.02, '100 km',
            transform=ax.transAxes, fontsize=10, fontweight='bold',
            ha='center', va='top', zorder=15)

add_scale_bar(ax1)
add_scale_bar(ax2)

print("✓ Scale bars and north arrows added")

fig.suptitle(
    'Spatial Density Comparison — Onshore vs Offshore Wind (Operational)\n' +
    'Projection: British National Grid (EPSG:27700)',
    fontsize=16,
    fontweight='bold',
    y=0.98
)

plt.tight_layout(rect=[0, 0, 1, 0.96])

# Step 6: Save Outputs
print("\nStep 6: Saving outputs...")

output_png = os.path.join(OUTPUT_DIR, 'map3_onshore_offshore_density_comparison.png')
plt.savefig(output_png, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ PNG map saved: {output_png}")

output_pdf = os.path.join(OUTPUT_DIR, 'map3_onshore_offshore_density_comparison.pdf')
plt.savefig(output_pdf, dpi=300, bbox_inches='tight', facecolor='white')
print(f"✓ PDF map saved: {output_pdf}")

plt.close()

# Step 7: Generate Summary Statistics
print("\nStep 7: Generating summary statistics...")

summary_data = {
    'Technology': ['Onshore', 'Offshore'],
    'Operational Farms': [len(onshore_gdf), len(offshore_gdf)],
    'Total Capacity (MW)': [
        onshore_gdf['Installed Capacity (MWelec)'].sum(),
        offshore_gdf['Installed Capacity (MWelec)'].sum()
    ],
    'Average Capacity (MW)': [
        onshore_gdf['Installed Capacity (MWelec)'].mean(),
        offshore_gdf['Installed Capacity (MWelec)'].mean()
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df['Total Capacity (GW)'] = summary_df['Total Capacity (MW)'] / 1000
summary_df = summary_df.round(2)

print("\n" + "="*80)
print("DENSITY COMPARISON SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)

summary_csv = os.path.join(OUTPUT_DIR, 'map3_density_summary.csv')
summary_df.to_csv(summary_csv, index=False)
print(f"\n✓ Summary statistics saved: {summary_csv}")

print("\n" + "="*80)
print("SPATIAL PATTERN INTERPRETATION")
print("="*80)

print("\nONSHORE WIND PATTERNS:")
print("  • Onshore wind farms show high concentration in Scotland")
print("  • Secondary clusters in Northern England and Wales")
print("  • Lower density in Southern and Central England")
print("  • Pattern follows wind resource availability and land availability")

print("\nOFFSHORE WIND PATTERNS:")
print("  • Offshore wind farms concentrated along the East Coast")
print("  • Significant clusters in the North Sea")
print("  • Scattered development in Irish Sea (west coast)")
print("  • Pattern follows shallow water depths and grid connection points")

print("\nKEY DIFFERENCES:")
print("  • Onshore: Dispersed across land with regional clustering")
print("  • Offshore: Linear patterns following coastline")
print("  • Onshore: More numerous but smaller average capacity")
print("  • Offshore: Fewer but larger average capacity")

print("="*80)

print("\n" + "="*80)
print("✓ Map 3 generation completed successfully!")
print("="*80)
print(f"\nOutputs saved to: {OUTPUT_DIR}")
print("  • map3_onshore_offshore_density_comparison.png")
print("  • map3_onshore_offshore_density_comparison.pdf")
print("  • map3_density_summary.csv")
print("\n" + "="*80)

Map 3: Spatial Density Comparison - Onshore vs Offshore Wind
Hexbin Density Analysis for Operational Wind Farms

Input directory:  D:/BENV0093/
Output directory: D:/BENV0093/outputs/

Step 1: Loading UK boundary shapefile...
✓ UK boundary loaded and dissolved
  CRS: EPSG:27700 (British National Grid)

Step 2: Loading renewable energy planning database...
✓ Total records loaded: 13,524
✓ Wind farms (onshore + offshore): 2,758

Step 3: Filtering operational wind farms...
✓ Operational wind farms: 827
  Onshore operational: 779
  Offshore operational: 48

Step 4: Processing spatial coordinates...
  Onshore: 779 with valid coordinates
  Offshore: 48 with valid coordinates

Step 5: Creating hexbin density comparison maps...
✓ Onshore density map created (logarithmic scale)
✓ Offshore density map created (linear scale)
✓ Scale bars and north arrows added

Step 6: Saving outputs...
✓ PNG map saved: D:/BENV0093/outputs/map3_onshore_offshore_density_comparison.png
✓ PDF map saved: D:/BENV0093/o